In [2]:
import pandas as pd

In [2]:
# Data from InsideAirBNB

df_calendar = pd.read_csv('Downloads/calendar.csv')
df_listing = pd.read_csv('Downloads/listings.csv')

df_calendar.columns = ['id', 'date', 'available', 'price']
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y-%m-%d')

df_calendar = df_calendar.loc[df_calendar['available'] == 'f']

agg_duration = df_calendar.groupby(['id']).size().reset_index()
agg_duration.columns = ['id', 'duration']

df_final = df_listing.merge(agg_duration, on='id', how='left')
df_final.to_csv('Downloads/offre_durationbyID.csv', sep=";", index=False)

/Users/wngauv/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (43,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
# Data from OpenData
df_loyer = pd.read_csv('Downloads/encadrement_loyers.csv')
df_loyer = df_loyer.loc[df_loyer['ville'] == 'PARIS']
df_loyer = df_loyer.loc[df_loyer['annee'] >= 2017]

In [4]:
# Data from OpenStreetMap
df_quartier = pd.read_csv('Downloads/osm_paris_quartiers.csv', sep=";", usecols=["osm_id", "name",
                                     "addr.postcode", "admin_level",
                                     "alt_name", "boundary",
                                     "ref.INSEE", "short_name", "type",
                                     "wikidata", "wikipedia"])

In [1]:
df_final = pd.read_csv('Downloads/offre_durationbyID.csv', sep=";")

NameError: name 'pd' is not defined